# Visualizaciones dinámicas

Este notebook compila una serie de pasos para convertir coordenadas en formato texto a coordenadas geográficas y seleccionar métricas en lapsos de tiempo para evaluar el comportamiento de las estaciones de subte. La visualización dinámica la realizaremos con la clase `scattermapbox` de la librería plotly express.

In [1]:
import pandas as pd

In [2]:
# construyamos el path de cada uno en base al año
paths = ['https://storage.googleapis.com/python_mdg/data_cursos/bici{}_cdn.csv'.format(str(y)) for y in range(15,20)] 

# y guardemos cada df bajo una key
df = {}
for p in paths:
    df[p[58:60]] = pd.read_csv(p)

In [4]:
!pip install plotly_express
import plotly_express as px

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Empecemos por construir una función que devuelva un dataframe con series de tipo datetime.

In [7]:
def atributos_temporales(dict_input,key):
    """
    Agrega columnas con diferentes atributos temporales 
    en un dataframe de pandas.
    ...
    Argumentos:
     dict_input (dict): diccionario de dataframes con viajes por año.
     key (str): año
     
    Devuelve:
      pandas.dataframe : df con series de mes,fecha,hora y dia.  
    """
    
    df = dict_input[key].copy()
    df['mes'] = pd.to_datetime(df.bici_Fecha_hora_retiro).dt.month
    df['fecha'] = pd.to_datetime(df.bici_Fecha_hora_retiro).dt.date.astype('datetime64') 
    df['hora'] = pd.to_datetime(df.bici_Fecha_hora_retiro).dt.hour
    df['dia_semana'] = pd.to_datetime(df.bici_Fecha_hora_retiro).dt.weekday
    df['nombre_dia_semana'] = df['dia_semana'].replace({0:'lunes', 
                                                        1:'martes', 
                                                        2:'miercoles', 
                                                        3:'jueves', 
                                                        4:'viernes', 
                                                        5:'sabado', 
                                                        6:'domingo'})
                         
    return df


Ahora armemos un constructor de filtros...

In [8]:
def construye_filtros(df, filtro):
    """
    Filtra un dataframe de pandas a partir de 
    los atributos temporales especificados en un dict.
    ...
    Argumentos:
        df (dataframe): pandas DataFrame de ecobici
        filtro (dict): Diccionario con dtypes key/str, val/list (e.g: {'mes':[6,12]})
        
    Devuelve:
      pandas.dataframe : df filtrado por mes o fecha. 
    """
    
    
    for k,v in filtro.items():
        if len(v) > 1:
            print('Filtrando df por {}, entre {} y {}'.format(k,v[0],v[1]))
            df1 = df[(df[k] >= v[0]) & (df[k] <= v[1])]
            return df1

        elif len(v) == 1:
            print('Filtrando df para {} {}'.format(k,v[0]))
            df1 = df[(df[k] == v[0])]
            return df1

        else:
            print('No se aplica ningún filtro')

E integremos ambos pasos en una misma función.

In [9]:
def compila_data(dict_input,key, filtro=None):
    '''
    Asigna atributos temporales a un df y filtra casos
    en función de los mismos.
    ...
    Argumentos:
    dict_input (dict): diccionario de dataframes con viajes por año
    key (str): año (e.g. '18')
    filtro (dict): diccionario con dtypes key/str, val/list (e.g: {'mes':[6,12]})
    
    Devuelve:
      pandas.dataframe : df de viajes origen/destino para un año determinado. 
    '''
    
    df = atributos_temporales(dict_input, key)
    
    if filtro:
        df_filt = construye_filtros(df,filtro)
        return df_filt
    else:
        return df

In [10]:
len(df['18'])

2970479

Ahora, apliquemos nuestra función de compilación. Tengamos en cuenta que la misma se aplicará sobre un dataframe de alrededor de unas 3 millones de filas. Es decir, bastante grande. Instanciemos y veamos qué tiempo insume...

In [11]:
%%time
t = compila_data(dict_input=df, key='18')

CPU times: user 5.88 s, sys: 205 ms, total: 6.08 s
Wall time: 6.24 s


In [12]:
t.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2970479 entries, 0 to 2970478
Data columns (total 14 columns):
 #   Column                        Dtype         
---  ------                        -----         
 0   bici_id_usuario               float64       
 1   bici_Fecha_hora_retiro        object        
 2   bici_tiempo_uso               float64       
 3   bici_nombre_estacion_origen   object        
 4   bici_estacion_origen          int64         
 5   bici_nombre_estacion_destino  object        
 6   bici_estacion_destino         float64       
 7   bici_sexo                     object        
 8   bici_edad                     float64       
 9   mes                           int64         
 10  fecha                         datetime64[ns]
 11  hora                          int64         
 12  dia_semana                    int64         
 13  nombre_dia_semana             object        
dtypes: datetime64[ns](1), float64(4), int64(4), object(5)
memory usage: 317.3+ MB


In [13]:
t.head()

,bici_id_usuario,bici_Fecha_hora_retiro,bici_tiempo_uso,bici_nombre_estacion_origen,bici_estacion_origen,bici_nombre_estacion_destino,bici_estacion_destino,bici_sexo,bici_edad,mes,fecha,hora,dia_semana,nombre_dia_semana
0,107148.0,2018-12-31 23:56:42,39.0,Parque Lezama,6,Austria y French,200.0,MASCULINO,27.0,12,2018-12-31,23,0,lunes
1,140759.0,2018-12-31 23:40:14,7.0,Cementerio de la Recoleta,166,Quintana,115.0,FEMENINO,65.0,12,2018-12-31,23,0,lunes
2,391310.0,2018-12-31 23:31:39,2.0,Agüero,85,Ecuador,69.0,MASCULINO,24.0,12,2018-12-31,23,0,lunes
3,487229.0,2018-12-31 23:27:19,31.0,Ministro Carranza,58,Sarandí,77.0,MASCULINO,21.0,12,2018-12-31,23,0,lunes
4,192675.0,2018-12-31 23:22:29,21.0,Pasco,91,Rincón,106.0,FEMENINO,32.0,12,2018-12-31,23,0,lunes


In [14]:
# revisemos si el dataframe sin filtros tiene la misma extensión que el original
len(t) == len(df['18'])

True

In [15]:
%%time
t1 = compila_data(dict_input=df, key='18', filtro={'mes':[12]})

Filtrando df para mes 12
CPU times: user 4.92 s, sys: 364 ms, total: 5.28 s
Wall time: 5.33 s


In [16]:
# filtrando para el mes de diciembre, vemos que la cantidad de casos es menor. Algo esperable...
len(t1)

166585

In [17]:
t1.head()

,bici_id_usuario,bici_Fecha_hora_retiro,bici_tiempo_uso,bici_nombre_estacion_origen,bici_estacion_origen,bici_nombre_estacion_destino,bici_estacion_destino,bici_sexo,bici_edad,mes,fecha,hora,dia_semana,nombre_dia_semana
0,107148.0,2018-12-31 23:56:42,39.0,Parque Lezama,6,Austria y French,200.0,MASCULINO,27.0,12,2018-12-31,23,0,lunes
1,140759.0,2018-12-31 23:40:14,7.0,Cementerio de la Recoleta,166,Quintana,115.0,FEMENINO,65.0,12,2018-12-31,23,0,lunes
2,391310.0,2018-12-31 23:31:39,2.0,Agüero,85,Ecuador,69.0,MASCULINO,24.0,12,2018-12-31,23,0,lunes
3,487229.0,2018-12-31 23:27:19,31.0,Ministro Carranza,58,Sarandí,77.0,MASCULINO,21.0,12,2018-12-31,23,0,lunes
4,192675.0,2018-12-31 23:22:29,21.0,Pasco,91,Rincón,106.0,FEMENINO,32.0,12,2018-12-31,23,0,lunes


Por último, veamos un filtro un poco más complejo.

In [18]:
%%time
t2 = compila_data(dict_input=df, key='18', filtro={'mes':[6,12]})

Filtrando df por mes, entre 6 y 12
CPU times: user 5.06 s, sys: 147 ms, total: 5.21 s
Wall time: 5.26 s


Sabemos entonces que en un dataframe de las características del nuestro, el compilador puede llegar a tardar hasta unos 5 segundos aproximadamente. Este último filtro nos sirve para seleccionar la cantidad de casos con un criterio de rango. Es decir, si la lista que nosotros pasamos en el parámetro contiene más de un elemento, esto será interpretado como que el primero de ellos es el límite inferior y el segundo el superior.

In [19]:
len(t2)

2074652

In [20]:
# al pasarle una lista con 6 y 12, nuestros datos corresponderan a los meses que se encuentran entre ambos límites 
t2.mes.unique()

array([12, 11, 10,  9,  8,  7,  6])

In [21]:
t3 = compila_data(dict_input=df, key='18',filtro={'fecha':['2018-12-31']})

Filtrando df para fecha 2018-12-31


In [22]:
len(t3)

1612

In [23]:
# acá filtramos entre los días 24 y 31 del mes de diciembre
t4 = compila_data(dict_input=df, key='18',filtro={'fecha':['2018-12-24','2018-12-31']})

Filtrando df por fecha, entre 2018-12-24 y 2018-12-31


In [24]:
len(t4)

25981

Utilicemos el compilador que construimos como herramienta para visualizar la cantidad de usuarios o retiros de bicicleta en distintas franjas temporales. La idea del flujo de trabajo que venimos constuyendo, es que nos provea el insumo necesario para reconstruir un patrón territorial que refleje la intensidad de uso del servicio.

Por eso, una vez que construimos nuestro dataframe, debemos convertirlo a un formato geográfico que nos permita representarlo espacialmente. Retomemos algunos de los conceptos que vimos en las clases iniciales. Convirtamos nuestro dataframe en geodataframe:

In [25]:
!apt install libspatialindex-dev
!pip install rtree
!pip install geopandas

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following additional packages will be installed:
  libspatialindex-c4v5 libspatialindex4v5
The following NEW packages will be installed:
  libspatialindex-c4v5 libspatialindex-dev libspatialindex4v5
0 upgraded, 3 newly installed, 0 to remove and 49 not upgraded.
Need to get 555 kB of archives.
After this operation, 3,308 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex4v5 amd64 1.8.5-5 [219 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-c4v5 amd64 1.8.5-5 [51.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libspatialindex-dev amd64 1.8.5-5 [285 kB]
Fetched 555 kB in 2s (348 kB/s)
Selecting previously unselected package libspatialindex

In [26]:
from geopandas import GeoDataFrame
from shapely.geometry import Point

In [27]:
def construye_gdf(data, gdf_coord):
    '''
    Transforma el df de viajes origen/destino en un df de cantidad de usuarios/retiros de bicicletas
    por estación y hora. Convierte el df resultante en geodataframe..
    ...
    Argumentos:
    data (df): df de viajes origen/destino
    gdf_coord (gdf): gdf con las coordenadas de referencia.
    
    Devuelve:
      geopandas.geodataframe : gdf de retiros por estación y hora. 
    '''
    
    info = data.groupby(['bici_nombre_estacion_origen','hora'])['bici_sexo'].count().reset_index()
    marker = gdf_coord.to_crs(epsg=4326)
    gdf = pd.merge(marker,info, right_on='bici_nombre_estacion_origen', left_on='NOMBRE')
    
    gdf['x'], gdf['y'] = gdf.geometry.x, gdf.geometry.y
    
    gb = gdf.groupby(['NOMBRE', 'x', 'y','hora']).sum()
    geometry = [Point(xy) for xy in zip(gb.reset_index().x, gb.reset_index().y)]
    info_estaciones = gb.reset_index().drop(['x', 'y'], axis=1)
    info_estaciones_geo = GeoDataFrame(info_estaciones, crs="EPSG:4326", geometry=geometry)
    
    info_estaciones_geo['usuarios'] = info_estaciones_geo["bici_sexo"]
    
    return info_estaciones_geo

In [29]:
import geopandas as gpd

In [31]:
from google.colab import drive
drive.mount('/drive/')

Mounted at /drive/


In [32]:
# Shape de estaciones descargados de DataBA
estaciones = gpd.read_file('/drive/MyDrive/Gestion de ciudades/data/estaciones_de_bicicleta.zip')

In [33]:
t1_map = construye_gdf(t1, estaciones)

In [34]:
t1_map.head()

,NOMBRE,hora,NRO_EST,bici_sexo,geometry,usuarios
0,15 de Noviembre,0,105,6,POINT (-58.40096 -34.63398),6
1,15 de Noviembre,1,105,5,POINT (-58.40096 -34.63398),5
2,15 de Noviembre,2,105,6,POINT (-58.40096 -34.63398),6
3,15 de Noviembre,5,105,3,POINT (-58.40096 -34.63398),3
4,15 de Noviembre,6,105,7,POINT (-58.40096 -34.63398),7


In [35]:
def mapa_estaciones(gdf, fecha):
    '''
    Plotea la cantidad de retiros de bicicletas por estación y hora
    para un período de tiempo determinado.
    ...
    Argumentos:
    gdf(gdf): GeoDataFrame.
    fecha(str): Título del plot.
    
    Devuelve:
      geopandas.geodataframe : gdf de retiros por estación y hora. 
    '''

    input_gdf = gdf.sort_values(by='hora', ascending=True)
    
    fig = px.scatter_mapbox(input_gdf, 
                            lat=input_gdf.geometry.y,
                            lon=input_gdf.geometry.x,
                            hover_name="NOMBRE",                        
                            animation_frame="hora",
                            size="usuarios",
                            color_discrete_sequence=['#F05E23'],
                            opacity=0.9,
                            zoom=11, 
                            height=600)
    
    fig.update_layout(mapbox_style="carto-darkmatter",
                      title_text = 'Usuarios/Retiros de bicicletas por estacion<br>({})'.format(fecha),
                      showlegend = True)

    fig.update_layout(margin={"r":1,"t":75,"l":0,"b":0})
    fig.show()

In [36]:
# veamos cómo queda nuestro mapa con alguno de los filtros
mapa_estaciones(t1_map, 
                'Diciembre 2018')

In [37]:
t4_map = construye_gdf(t4, estaciones)

In [38]:
# como son menos casos, ordenamos la columna de horas para asegurarnos que el eje x se vea ordenado 
mapa_estaciones(t4_map.sort_values(by='hora'), "21 a 31 de diciembre 2018")

Y aquí, nuestro resultado final. Optimizando un poco más esta herramienta, podríamos monitorear la cantidad de retiros e identificar, por hora, cuáles son las zonas con mayor demanda.